In [ ]:
#A seizure forescasting system using physiological data
#Data preprocessing
import pandas as pd
import numpy as np

# Data segmentation (assuming sensor data is in a DataFrame 'df')
window_size = 30  # 30 seconds window
step_size = 5  # 5 seconds step

def segment_data(df, window_size, step_size):
    segments = []
    for i in range(0, len(df) - window_size, step_size):
        segments.append(df.iloc[i:i + window_size])
    return segments

# Usage:
segments = segment_data(df, window_size, step_size)


In [ ]:
#Extracting useful features like mean, standard deviation from the raw sensor data
from scipy.stats import skew, kurtosis
from numpy.fft import fft

def extract_features(window_data):
    features = {}
    features['mean'] = np.mean(window_data)
    features['std'] = np.std(window_data)
    features['skew'] = skew(window_data)
    features['kurtosis'] = kurtosis(window_data)
    
    # Frequency domain features using FFT to capture periodicity
    fft_vals = fft(window_data)
    features['fft_mean'] = np.mean(np.abs(fft_vals))
    features['fft_std'] = np.std(np.abs(fft_vals))
    
    return features

# Feature extraction
window_data = segments[0]  # Window data
features = extract_features(window_data['sensor_signal'].values)

In [ ]:
#Constructing an LSTM network with multiple layers to learn temporal patterns in the data
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
#After feature extraction, preparing data for training
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(64, activation='relu', input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))  # Output probability of seizure
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Usage:
input_shape = (time_steps, num_features)  # Define based on  data
model = build_lstm_model(input_shape)
model.summary()

In [ ]:
#Training the LSTM Model using the labelled data
history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_val, y_val))


In [ ]:
#Generate random baseline to evaluate the model
random_preds = np.random.randint(0, 2, size=y_test.shape)

# Evaluate performance
accuracy = accuracy_score(y_test, random_preds)
precision = precision_score(y_test, random_preds)
recall = recall_score(y_test, random_preds)
f1 = f1_score(y_test, random_preds)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
